## Original code 

In [ ]:
import pennylane as qml
from pennylane import qchem
from pennylane import numpy as np
from itertools import chain
import time
import re
from scipy.optimize import minimize
ash_excitation = []
energies = []
excitations= []

X = qml.PauliX
Y = qml.PauliY
Z = qml.PauliZ
I = qml.Identity



bond_length = 3.0
symbols = ["H","H","H","H"]
electrons = 4
orbitals = 8
r_bohr = bond_length *1.8897259886  
coordinates = np.array([[0.0,0.0, 1*r_bohr], [0.0, 0.0, 2*r_bohr], [0.0,0.0,3*r_bohr],[0.0, 0.0, 4*r_bohr]])
H, qubits = qml.qchem.molecular_hamiltonian(symbols, coordinates, basis="sto-6g", method="pyscf")
hf_state = qchem.hf_state(electrons, qubits)




def adaptvqe(adapt_it = 4, e_th=1e-12):
    #Calculation of HF state
    dev = qml.device("lightning.qubit", wires=qubits)
    @qml.qnode(dev)
    def circuit(hf_state, electrons, qubits, H):
        print('Updated hf_state is', hf_state)  
        qml.BasisState(hf_state, wires=range(qubits))
        return qml.expval(H)   #Calculating the expectation value of the Hamiltonian
    
    # Commutator calculation for HF state
    @qml.qnode(dev)
    def commutator_0(H,w, k):  #H is the Hamiltonian, w is the operator, k is the basis state - HF state
        qml.BasisState(k, wires=range(qubits))
        res = qml.commutator(H, w)   #Calculating the commutator
        return qml.expval(res)
    
    # Commutator calculation for other states except HF state
    @qml.qnode(dev)
    def commutator_1(H,w, k): #H is the Hamiltonian, w is the operator, k is the basis state
        qml.StatePrep(k, wires=range(qubits))
        res = qml.commutator(H, w) #Calculating the commutator
        return qml.expval(res)

    #Energy calculation 
    @qml.qnode(dev)
    def ash(params, ash_excitation, hf_state, H):
        [qml.PauliX(i) for i in np.nonzero(hf_state)[0]]  #Appln of HF state
        for i, excitation in enumerate(ash_excitation):
            if len(ash_excitation[i]) == 4:
                qml.FermionicDoubleExcitation(weight=params[i], wires1=ash_excitation[i][2:][::-1], wires2=ash_excitation[i][:2][::-1])
            elif len(ash_excitation[i]) == 2:
                qml.FermionicSingleExcitation(weight=params[i], wires=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)))
        return qml.expval(H)  #Calculating the expectation value of the Hamiltonian
    
    # Calculation of New state, same as the above function but with the state return
    dev1 = qml.device("lightning.qubit", wires=qubits)
    @qml.qnode(dev1)
    def new_state(hf_state, ash_excitation, params):
        [qml.PauliX(i) for i in np.nonzero(hf_state)[0]] #Applying the HF state
        for i, excitations in enumerate(ash_excitation):
            if len(ash_excitation[i]) == 4:
                print('Exc. dealing right now is', ash_excitation[i])
                print('The params that are going in', params[i])
                qml.FermionicDoubleExcitation(weight=params[i], wires1=ash_excitation[i][2:][::-1], wires2=ash_excitation[i][:2][::-1])
            elif len(ash_excitation[i]) == 2:
                print('Single Exc. dealing right now is', ash_excitation[i])
                print('Single exc params that are going in', params[i])
                qml.FermionicSingleExcitation(weight=params[i], wires=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)))
        return qml.state()
    
    def cost(params):
        energy = ash(params, ash_excitation, hf_state, H)
        return energy

    def callback(params):
        print(f"Current parameters: {params}")
        print(f"Current cost: {cost(params)}\n")
    

    print('HF state is', circuit(hf_state, electrons, qubits, H))
    singles, doubles = qml.qchem.excitations(electrons, orbitals)
    op1 =  [qml.fermi.FermiWord({(0, x[0]): "+", (1, x[1]): "-"}) for x in singles]
    op2 =  [qml.fermi.FermiWord({(0, x[0]): "+", (1, x[1]): "+", (2, x[2]): "-", (3, x[3]): "-"})for x in doubles]
    operator_pool = (op1) + (op2)  #Operator pool - Singles and Doubles
    states = [hf_state]
    

    for j in range(adapt_it):
        print('The adapt iteration now is', j)  #Adapt iteration
        max_value = float('-inf')
        max_operator = None
        # Start with the most recent state (last state in the list)
        k = states[-1] if states else hf_state  # if states is empty, fall back to hf_state
       
        for i in operator_pool:
            print('The current excitation operator is', i)   #Current excitation operator - fermionic one
            w = qml.fermi.jordan_wigner(i)  #JW transformation
            if np.array_equal(k, hf_state): # If the current state is the HF state
                print('Print, if this is activated - HF state')
                current_value = abs(2*(commutator_0(H, w, k)))      #Commutator calculation is activated  
            else:
                current_value = abs(2*(commutator_1(H, w, k)))      #For other states, commutator calculation is activated
            print(f'The expectation value of {i} is', current_value)

            if current_value > max_value:
                max_value = current_value
                max_operator = i

        print(f"The highest operator value is {max_value} for operator {max_operator}")  #Highest operator value

        # Convert operator to excitations and append to ash_excitation
        indices_str = re.findall(r'\d+', str(max_operator))
        excitations = [int(index) for index in indices_str]
        print('Highest gradient excitation is', excitations)
        ash_excitation.append(excitations) #Appending the excitations to the ash_excitation
        print('The current status of ash_excitation is', ash_excitation)
        print('Moving towards parameters')
        params = np.zeros(len(ash_excitation), requires_grad=True)  #Parameters initialization
        print('The length of parameters is', params)


        #Energy calculation
        result = minimize(cost, params, method='powell', callback=callback, tol = e_th, options = {'disp': True, 'maxiter': 1e8})

        print("Final updated parameters:", result.x)
        print("Final cost:", result.fun)

        params= (result.x)
        energies.append(result.fun)
        print('Updated params are', params)
        print('Updated excitation are', ash_excitation)

        # New state generation - With the updated parameters
        ostate = new_state(hf_state, ash_excitation, params)
        print(qml.draw(new_state, max_length=100)(hf_state,ash_excitation,params))
        # Append the new state to the states list
        states.append(ostate)

    return energies, params, ash_excitation




## Introduced class of AdaptVQE

In [ ]:
import pennylane as qml
import numpy as np
import re
from scipy.optimize import minimize
from pennylane import qchem

class AdaptVQESri:
    def __init__(self, adapt_it, bond_length, e_th, electrons, symbols=["H","H","H","H"], orbitals=8):
        
        self.e_th = e_th
        #self.shots = shots
        # Initialize the class with parameters
        self.bond_length = bond_length
        self.symbols = symbols
        self.electrons = electrons
        self.orbitals = orbitals
        self.adapt_it = adapt_it
        
        # Convert bond length to bohr
        self.r_bohr = bond_length * 1.8897259886  
        # Set up coordinates
        self.coordinates = np.array([[0.0,0.0, 1*self.r_bohr], [0.0, 0.0, 2*self.r_bohr], [0.0,0.0,3*self.r_bohr],[0.0, 0.0, 4*self.r_bohr]])
        
        # Set up the Hamiltonian and qubits
        self.H, self.qubits = qml.qchem.molecular_hamiltonian(self.symbols, self.coordinates, basis="sto-6g", method="pyscf")
        self.hf_state = qchem.hf_state(self.electrons, self.qubits)
        
        # Initial variables
        self.ash_excitation = []
        self.energies = []

    def adapt_vqe(self):
        dev = qml.device("lightning.qubit", wires=self.qubits, shots = None)
        @qml.qnode(dev)
        def circuit(hf_state, electrons, qubits, H):
            # Prepare the Hartree-Fock state
            qml.BasisState(hf_state, wires=range(qubits))
            return qml.expval(H)   #Calculating the expectation value of the Hamiltonian

        @qml.qnode(dev)
        def commutator_0(H, w, k):
            qml.BasisState(k, wires=range(self.qubits))
            res = qml.commutator(H, w)
            return qml.expval(res)
        
        @qml.qnode(dev)
        def commutator_1(H, w, k):
            qml.StatePrep(k, wires=range(self.qubits))
            res = qml.commutator(H, w)
            return qml.expval(res)

        @qml.qnode(dev)
        def ash(params, ash_excitation, hf_state, H):
            [qml.PauliX(i) for i in np.nonzero(hf_state)[0]]
            for i, excitation in enumerate(ash_excitation):
                if len(ash_excitation[i]) == 4:
                    qml.FermionicDoubleExcitation(weight=params[i], wires1=ash_excitation[i][2:][::-1], wires2=ash_excitation[i][:2][::-1])
                elif len(ash_excitation[i]) == 2:
                    qml.FermionicSingleExcitation(weight=params[i], wires=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)))
            return qml.expval(H)

        def cost(params):
            energy = ash(params, self.ash_excitation, self.hf_state, self.H)
            return energy
        
        def callback(params):
            print(f"Current parameters: {params}")
            print(f"Current cost: {cost(params)}\n")

        print('HF state is', circuit(self.hf_state, self.electrons, self.qubits, self.H))
        
        singles, doubles = qml.qchem.excitations(self.electrons, self.orbitals)
        op1 = [qml.fermi.FermiWord({(0, x[0]): "+", (1, x[1]): "-"}) for x in singles]
        op2 = [qml.fermi.FermiWord({(0, x[0]): "+", (1, x[1]): "+", (2, x[2]): "-", (3, x[3]): "-"}) for x in doubles]
        operator_pool = op1 + op2
        states = [self.hf_state]

        for j in range(self.adapt_it):
            print(f'The adapt iteration now is {j}')
            max_value = float('-inf')
            max_operator = None
            k = states[-1] if states else self.hf_state

            for i in operator_pool:
                w = qml.fermi.jordan_wigner(i)
                if np.array_equal(k, self.hf_state):
                    current_value = abs(2*(commutator_0(self.H, w, k)))
                else:
                    current_value = abs(2*(commutator_1(self.H, w, k)))
                if current_value > max_value:
                    max_value = current_value
                    max_operator = i

            print(f"The highest operator value is {max_value} for operator {max_operator}")

            indices_str = re.findall(r'\d+', str(max_operator))
            excitations = [int(index) for index in indices_str]
            self.ash_excitation.append(excitations)
            
            print('Ash excitation right now is',self.ash_excitation)
            params = qml.numpy.zeros(len(self.ash_excitation), requires_grad=True)

            result = minimize(cost, params, method='powell', callback=callback, tol=self.e_th, options={'disp': True, 'maxiter': 1e8})

            print("Final updated parameters:", result.x)
            print("Final cost:", result.fun)

            params = result.x
            self.energies.append(result.fun)

            ostate = self.new_state(self.hf_state, self.ash_excitation, params)
            print(qml.draw(self.new_state, max_length=100)(self.hf_state, self.ash_excitation, params))
            states.append(ostate)

        return self.energies,  params, self.ash_excitation

    def new_state(self, hf_state, ash_excitation, params):
        dev1 = qml.device("lightning.qubit", wires=self.qubits)
        @qml.qnode(dev1)
        def new_state_qnode(hf_state, ash_excitation, params):
            [qml.PauliX(i) for i in np.nonzero(hf_state)[0 ]]
            for i, excitations in enumerate(ash_excitation):
                if len(ash_excitation[i]) == 4:
                    qml.FermionicDoubleExcitation(weight=params[i], wires1=ash_excitation[i][2:][::-1], wires2=ash_excitation[i][:2][::-1])
                elif len(ash_excitation[i]) == 2:
                    qml.FermionicSingleExcitation(weight=params[i], wires=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)))
            return qml.state()
        
        return new_state_qnode(hf_state, ash_excitation, params)


In [ ]:
# Create an instance of AdaptVQE class
adapt_it = 4
e_th = 1e-8
bond_length = 5.0
electrons= 4
shots = 0.0


vqe_instance = AdaptVQESri(adapt_it=adapt_it, bond_length=bond_length, e_th=e_th, electrons=electrons)



# Call the adapt_vqe method to perform the calculation
energies,params, ash_excitation = vqe_instance.adapt_vqe()

# You can access any class attribute like this:
print(f"Final energies: {energies}")
print(f"Final parameters: {params}")


## Procedure to call the class 

In [ ]:
import ipynb
from ipynb.fs.full.AdaptVQEclass import AdaptVQESri   
#The file name is AdaptVQEclass.ipynb and in that file, I am importing the class called AdaptVQESri

adapt_it = 4
e_th = 1e-8
bond_length = 3.0
electrons= 4



vqe_instance = AdaptVQESri(adapt_it=adapt_it, bond_length=bond_length, e_th=e_th, electrons=electrons)



# Call the adapt_vqe method to perform the calculation
energies,params, ash_excitation = vqe_instance.adapt_vqe()

# You can access any class attribute like this:
print(f"Final energies: {energies}")
print(f"Final parameters: {params}")